# Smoke

[![Google Collab Book](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/tum-pbs/PhiFlow/blob/develop/examples/grids/Smoke_Plume.ipynb)

This example simulates a rising plume of hot smoke. It demonstrates how to advect and couple other quantities with the fluid.

In [9]:
%pip install phiflow
#from phi.jax.flow import *
from phi.torch.flow import *
from tqdm.notebook import trange
print(phi.__version__)
import torch
print(torch.__version__)
# from phi.flow import *  # If JAX is not installed. You can use phi.torch or phi.tf as well.

3.1.0
2.5.0+cu121


We begin by defining our simulation size and smoke inflow.

In [10]:
domain = Box(x=100, y=100)
inflow = Sphere(x=50, y=9.5, radius=5)
inflow_rate = 0.2

Our simulation advects the velocity and smoke fields, computes the buoyancy force proportional to the smoke density, and enforces incompressibility.
We sample the velocities on a $64\times 64$ staggered grid, and the smoke density on a $200\times 200$ grid.

In [11]:
@jit_compile
def step(v, s, p, dt):
    s = advect.mac_cormack(s, v, dt) + inflow_rate * resample(inflow, to=s, soft=True)
    buoyancy = resample(s * (0, 0.1), to=v)
    v = advect.semi_lagrangian(v, v, dt) + buoyancy * dt
    v, p = fluid.make_incompressible(v, (), Solve('CG', 1e-3, x0=p))
    return v, s, p

v0 = StaggeredGrid(0, 0, domain, x=64, y=64)
smoke0 = CenteredGrid(0, ZERO_GRADIENT, domain, x=200, y=200)
print(ZERO_GRADIENT)

zero-gradient


In [8]:
v_trj, s_trj, p_trj = iterate(step, batch(time=30), v0, smoke0, None, dt=.5, range=trange, substeps=3)

  0%|          | 0/30 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/phiml/backend/torch/_torch_backend.py:800: UserWarning: Sparse CSR tensor support is in beta state. If you miss a functionality in the sparse tensor support, please submit a feature request to https://github.com/pytorch/pytorch/issues. (Triggered internally at ../aten/src/ATen/SparseCsrTensorImpl.cpp:53.)
  return torch.sparse_csr_tensor(row_pointers, column_indices, values, shape, device=values.device)


In [13]:
plot(s_trj, animate='time', frame_time=80)
